In [1]:
import sqlite3
import os
import re
from pathlib import Path
from markupsafe import Markup

In [2]:
%%time

search_text = "kutya"
case_sensitive = False
exact_match = False

def fetch_data_from_db(database_path, query):
    with sqlite3.connect(database_path) as conn:
        cursor = conn.cursor()
        cursor.execute(query)
        rows = cursor.fetchall()
        columns = [desc[0] for desc in cursor.description]
    return [dict(zip(columns, row)) for row in rows]  # Returns a list of dictionaries.

DATABASE_PATH = Path("onkorm.db")
txt_folder = Path("../txt")

flags = 0 if case_sensitive else re.IGNORECASE
pattern = rf"\b{re.escape(search_text)}\b" if exact_match else re.escape(search_text)
regex = re.compile(pattern, flags)

mappa_query = "SELECT * FROM ujbuda_meghivo_mappa"
reszlet_query = "SELECT * FROM ujbuda_meghivo_reszlet"
napi_query = "SELECT * FROM ujbuda_napirendi"

mappa_data = fetch_data_from_db(DATABASE_PATH, mappa_query)
reszlet_data = fetch_data_from_db(DATABASE_PATH, reszlet_query)
napi_data = fetch_data_from_db(DATABASE_PATH, napi_query)

results = []

for folder in os.listdir(txt_folder):
    folder_path = os.path.join(txt_folder, folder)

    for subfolder in os.listdir(folder_path):
        subfolder_path = os.path.join(folder_path, subfolder)

        for file in os.listdir(subfolder_path):
            file_path = os.path.join(subfolder_path, file)

            with open(file_path, 'r', encoding='utf-8') as f:
                lines = f.readlines()

            for i, line in enumerate(lines):
                if regex.search(line):
                    start, end = max(0, i - 3), min(len(lines), i + 4)
                    context = ''.join(lines[start:end]).strip()
                    highlighted_context = re.sub(
                        pattern,
                        lambda m: f"<span class='highlight'>{m.group(0)}</span>",
                        context,
                        flags=flags,
                    )

                    napi_entry = next((n for n in napi_data if n["uuid"] == subfolder), None)
                    if napi_entry:

                        detail_entry = next((r for r in reszlet_data if r["folder_uuid"] == folder), None)

                        if detail_entry:
                            results.append({
                                'dátum': detail_entry["datum"],
                                'név': detail_entry["nev"],
                                'napirendi pont': napi_entry["targy"],
                                'file': file,
                                'context': Markup(highlighted_context),
                            })
                    break

print(f"Number of results: {len(results)}")



Number of results: 17
CPU times: user 305 ms, sys: 70.9 ms, total: 376 ms
Wall time: 376 ms


{'dátum': '2023.09.14.',
 'név': 'Képviselő-testület',
 'napirendi pont': 'Településrendezési szerződések megkötése',
 'file': 'Telepítési tanulmányterv Hosszúréti utca.txt',
 'context': Markup("Az egykor gyümölcsösként használt területet mára már erdőszerű, spontán nőtt fás növényzet \nborítja. A fa és cserjeállomány néhány idősebb, értékes faegyedből, valamint pionír fa és \ncserjefajok sűrűn benőtt zárt tömbjeiből, továbbá kisebb füves nyiladékokból áll. A területet \njelenleg a környező, már beépült lakóterületek lakossága <span class='highlight'>kutya</span>sétáltatásra használja. \n \n\x0c11")}

In [4]:
from datetime import datetime


'2023.09.14.'

In [7]:
datetime.strptime(results[0]['dátum'], '%Y.%m.%d.')

datetime.datetime(2023, 9, 14, 0, 0)